In [31]:
import urllib.request, json 
import pandas as pd
import numpy as np


In [32]:
df = pd.read_csv("./IPL_Deliveries_Full.csv")
df.shape

(136598, 21)

In [33]:
df.dtypes

match_id             int64
inning               int64
batting_team        object
bowling_team        object
over                 int64
ball                 int64
batsman             object
non_striker         object
bowler              object
is_super_over        int64
wide_runs            int64
bye_runs             int64
legbye_runs          int64
noball_runs          int64
penalty_runs         int64
batsman_runs         int64
extra_runs           int64
total_runs           int64
player_dismissed    object
dismissal_kind      object
fielder             object
dtype: object

In [263]:
#Build a dictionary of Matches player by each batsman
played = {}
def BuildPlayedDict(x):
    #print(x.shape, x.shape[0], x.shape[1])
    for p in x.batsman.unique():
        if p in played:
            played[p] += 1
        else:
            played[p] = 1

df.groupby('match_id').apply(BuildPlayedDict)
import operator
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
#sorted_x = sorted(x.items(), key=operator.itemgetter(1))
#played_sorted = sorted(played.items(), key=operator.itemgetter(1), reverse = True)
#played_sorted

In [265]:
#average runs per ball, across the 20 overs, while batting 1st or 2nd, by teams
%matplotlib inline
df[df.inning.isin([1,2])].pivot_table(index = 'over', columns=['batting_team','inning'], values = 'total_runs', aggfunc=np.mean, margins= True)


batting_team Chennai Super Kings           Deccan Chargers            \
inning                         1         2               1         2   
over                                                                   
1                       0.843424  0.799410        0.996269  1.085859   
2                       1.075975  1.238938        1.261818  1.283654   
3                       1.279412  1.198251        1.136882  1.321782   
4                       1.142857  1.570588        1.107407  1.339901   
5                       1.487234  1.312500        1.172932  1.383838   
6                       1.419958  1.397590        1.209924  1.364078   
7                       1.077406  1.152047        1.011407  1.051020   
8                       1.160000  1.225146        1.167300  1.171717   
9                       1.201699  1.314286        1.132075  1.095000   
10                      1.129237  1.156740        1.083969  1.084577   
11                      1.269474  1.180952        1.287356  1.158163   
12                      1.409664  1.101538        1.350943  1.358974   
13                      1.282378  1.199377        1.191571  1.099476   
14                      1.381857  1.347962        1.262357  1.302198   
15                      1.443975  1.360759        1.389513  1.278689   
16                      1.430998  1.440000        1.471910  1.135593   
17                      1.631250  1.476667        1.651341  1.497238   
18                      1.630342  1.531690        1.714286  1.279762   
19                      1.733766  1.578125        1.513308  1.394737   
20                      1.798246  1.973404        1.530120  1.470588   
All                     1.339207  1.307128        1.281250  1.251465   

batting_team Delhi Daredevils           Gujarat Lions            \
inning                      1         2             1         2   
over                                                              
1                    0.901370  1.077406      1.000000  0.936508   
2                    1.077562  1.314465      1.135135  1.031250   
3                    1.266667  1.182979      0.972222  1.888889   
4                    1.419263  1.211538      0.837838  1.677419   
5                    1.261236  1.312896      1.675676  1.387097   
6                    1.126404  1.202151      1.000000  1.830769   
7                    1.019499  1.299127      1.128205  1.569231   
8                    1.037572  1.211663      1.270270  1.350000   
9                    1.158501  1.256466      1.052632  0.904762   
10                   1.048023  1.261438      1.421053  1.206349   
11                   1.367816  1.158458      1.138889  1.245902   
12                   1.135294  1.334802      0.972222  1.196721   
13                   1.227405  1.360619      1.189189  1.032787   
14                   1.229651  1.288288      1.111111  1.491525   
15                   1.171091  1.110345      1.250000  1.196429   
16                   1.299703  1.383495      1.153846  1.672727   
17                   1.332378  1.471939      1.459459  1.305085   
18                   1.581121  1.491379      1.447368  1.240741   
19                   1.721362  1.456311      1.684211  1.344828   
20                   1.785016  1.665116      1.378378  1.541667   
All                  1.251660  1.284901      1.215633  1.347258   

batting_team Kings XI Punjab              ...    Pune Warriors  \
inning                     1         2    ...                2   
over                                      ...                    
1                   0.881818  0.980392    ...         0.874214   
2                   1.135945  1.079327    ...         1.159509   
3                   1.206897  1.282555    ...         1.271523   
4                   1.244656  1.431421    ...         1.275641   
5                   1.311778  1.349127    ...         1.121795   
6                   1.368910  1.381773    ...         1.181818   
7                   1.023753  1.195489    ...         0.923077   
8 

In [231]:
#utility function used later
def trybuild(lookuplist, buildlist):
    alist = []
    for i in buildlist.index:
        try:
            #print(i)
            alist.append(lookuplist[i])
            #print(alist)
        except KeyError:
            #print('except')
            alist.append(0)
    return alist

In [283]:
BatsmanStats = df.groupby('batsman').aggregate({'ball': 'count', 'batsman_runs': 'sum'})
BatsmanStats.rename(columns={'ball': 'balls', 'batsman_runs': 'runs'}, inplace=True)
BatsmanStats['strike_rate'] = BatsmanStats['runs']/BatsmanStats['balls'] * 100
BatsmanStats['matches_played'] = [played[p] for p in BatsmanStats.index]
BatsmanStats['average']= BatsmanStats['runs']/BatsmanStats['matches_played']

for dk in df.dismissal_kind.unique():
    if dk in ['nan','hit wicket', 'retired hurt', 'obstructing the field']:
        continue
    lookuplist = df[df.dismissal_kind == dk].groupby('player_dismissed')['player_dismissed'].count()
    BatsmanStats[dk] = trybuild(lookuplist, BatsmanStats)

for r in df.batsman_runs.unique():
    lookuplist = df[df.batsman_runs == r].groupby('batsman')['batsman'].count()
    BatsmanStats[str(r) + 's'] = trybuild(lookuplist, BatsmanStats)

BatsmanStats['6s/match'] = BatsmanStats['6s']/BatsmanStats['matches_played']    
TopBatsman = BatsmanStats.sort_values(['balls','runs'], ascending=False)[:20]
TopBatsman


,balls,runs,strike_rate,matches_played,average,nan,caught,bowled,run out,lbw,stumped,caught and bowled,0s,4s,6s,1s,2s,5s,3s,6s/match
batsman,,,,,,,,,,,,,,,,,,,,
V Kohli,3237,4115,127.123880,132,31.174242,0,68,23,6,6,3,2,1192,361,149,1303,222,0,10,1.128788
SK Raina,3059,4106,134.226872,143,28.713287,0,87,9,10,3,7,7,1081,360,161,1227,219,1,10,1.125874
G Gambhir,3028,3634,120.013210,130,27.953846,0,66,25,15,7,2,4,1167,422,51,1151,222,0,15,0.392308
RG Sharma,2996,3874,129.305741,137,28.277372,0,77,17,9,4,3,5,1082,323,164,1261,163,1,2,1.197080
RV Uthappa,2721,3390,124.586549,130,26.076923,0,78,11,6,10,7,3,1090,341,104,990,182,3,11,0.800000
S Dhawan,2622,3082,117.543860,112,27.517857,0,54,18,13,5,4,2,1077,348,62,972,147,2,14,0.553571
DA Warner,2440,3373,138.237705,100,33.730000,0,56,17,6,3,4,2,950,338,134,830,177,0,11,1.340000
MS Dhoni,2419,3270,135.179826,128,25.546875,0,60,10,6,2,2,3,810,236,140,991,231,0,11,1.093750
CH Gayle,2360,3451,146.228814,91,37.923077,0,47,22,1,6,2,2,1088,282,252,668,67,0,3,2.769231


In [278]:
played_bowler = {}
def BuildPlayedBowlerDict(x):
    #print(x.shape, x.shape[0], x.shape[1])
    for p in x.bowler.unique():
        if p in played_bowler:
            played_bowler[p] += 1
        else:
            played_bowler[p] = 1

df.groupby('match_id').apply(BuildPlayedBowlerDict)
played_bowler

{'A Ashish Reddy': 20,
 'A Chandila': 12,
 'A Flintoff': 3,
 'A Kumble': 42,
 'A Mishra': 112,
 'A Mithun': 16,
 'A Nehra': 82,
 'A Nel': 1,
 'A Singh': 23,
 'A Symonds': 30,
 'A Uniyal': 2,
 'A Zampa': 5,
 'AA Chavan': 13,
 'AA Jhunjhunwala': 10,
 'AA Kazi': 1,
 'AA Noffke': 2,
 'AB Agarkar': 43,
 'AB Dinda': 73,
 'AB McDonald': 10,
 'AC Gilchrist': 1,
 'AC Thomas': 15,
 'AC Voges': 7,
 'AD Mascarenhas': 13,
 'AD Mathews': 42,
 'AD Russell': 34,
 'AF Milne': 1,
 'AG Murtaza': 12,
 'AJ Finch': 5,
 'AL Menaria': 12,
 'AM Nayar': 19,
 'AM Rahane': 1,
 'AM Salvi': 7,
 'AN Ahmed': 17,
 'AP Dole': 3,
 'AR Patel': 45,
 'AS Rajpoot': 6,
 'AS Raut': 5,
 'AUK Pathan': 3,
 'Abdur Razzak': 1,
 'Anand Rajan': 8,
 'Ankit Sharma': 19,
 'Anureet Singh': 18,
 'Azhar Mahmood': 23,
 'B Akhil': 13,
 'B Brainder Sran': 1,
 'B Chipli': 1,
 'B Geeves': 2,
 'B Kumar': 76,
 'B Laughlin': 2,
 'B Lee': 38,
 'BA Bhatt': 15,
 'BAW Mendis': 10,
 'BB Sran': 14,
 'BCJ Cutting': 5,
 'BE Hendricks': 7,
 'BJ Hodge': 20

In [282]:
bowlerByBall = df.groupby('bowler').aggregate({'ball': 'count', 'total_runs': 'sum', 'player_dismissed' : 'count'})
bowlerByBall.columns = ['balls','runs','wickets']
bowlerByBall['economy'] = bowlerByBall['runs']/bowlerByBall['balls'] * 6
bowlerByBall['strikerate'] = bowlerByBall['runs']/bowlerByBall['wickets']
bowlerByBall['matches_played'] = [played_bowler[p] for p in bowlerByBall.index]
bowlerByBall.sort_values(['wickets','economy'], ascending=False)
TopBowlerBywickets = bowlerByBall.sort_values(['wickets','economy'], ascending=False)[:20]
TopBowlerBywickets

,balls,runs,wickets,economy,strikerate,matches_played
bowler,,,,,,
SL Malinga,2407,2646,159,6.595762,16.641509,98
DJ Bravo,2110,2815,137,8.004739,20.547445,103
A Mishra,2466,2960,132,7.201946,22.424242,112
Harbhajan Singh,2742,3184,128,6.967177,24.875000,123
PP Chawla,2472,3136,127,7.611650,24.692913,122
R Vinay Kumar,2141,2945,123,8.253153,23.943089,101
A Nehra,1842,2339,111,7.618893,21.072072,82
R Ashwin,2359,2552,110,6.490886,23.200000,108
Z Khan,2030,2537,107,7.498522,23.710280,89
